<a href="https://colab.research.google.com/github/Jamess200/BirdnetProject/blob/main/Test_BirdNET_Audio_Cloud_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **<------------ Part 1 ------------>**

In [ ]:
#@title **Testing Alternative Method for Audio Processing and CSV Creation**
# This notebook guides you through analysing bird audio recordings using BirdNET.

In [ ]:
#@title **Step 1: Install & Import Required Libraries**
# Install PyDrive to download files from Google Drive, and other packages for processing.
!pip install PyDrive birdnetlib tflite-runtime resampy ffmpeg pygbif mplcursors

# Import necessary libraries for BirdNET
import resampy
import birdnetlib
import tflite_runtime
import ffmpeg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplcursors
import os
import re
from google.colab import drive
from google.colab import files
from sklearn.metrics import confusion_matrix, classification_report
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime
from pygbif import occurrences
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.7 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=792d34b02c896db3ee484579043b48459d1bdd130

In [ ]:
#@title **Step 2: Clone the GitHub Repository**
# This step clones the required GitHub repository into the Colab environment.

# Define the path where the repository will be cloned
repo_path = '/content/BirdnetProject'

# Check if the repository already exists
if not os.path.exists(repo_path):
    # Clone the repository if it doesn't exist
    !git clone https://github.com/Jamess200/BirdnetProject
else:
    # If the repository exists, pull the latest changes
    os.chdir(repo_path)
    !git pull

# Verify cloned repository
print("Repository contents:")
!ls /content/BirdnetProject

Cloning into 'BirdnetProject'...
remote: Enumerating objects: 748, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 748 (delta 37), reused 5 (delta 4), pack-reused 682 (from 1)
Receiving objects: 100% (748/748), 1.36 GiB | 20.07 MiB/s, done.
Resolving deltas: 100% (351/351), done.
Repository contents:
Files  README.md


In [ ]:
#@title **Step 3: Authenticate and Create PyDrive Client**
# This step authenticates with Google to access the shared folder on Google Drive.

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("Authentication successful. You can now access files from the shared Google Drive folder.")

Authentication successful. You can now access files from the shared Google Drive folder.


In [ ]:
#@title **Step 4: Define Helper Functions**
# These functions help to download the test audio file from my Google Drive folder to a local directory and list files.

def download_files_from_drive(folder_id, destination):
    """
    Download all files from a Google Drive folder to a local directory.

    :param folder_id: str, The ID of the Google Drive folder
    :param destination: str, The local directory to save the files
    """
    # List all files in the shared folder
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    # Create a directory to store the downloaded audio files
    os.makedirs(destination, exist_ok=True)

    # Download each file to the local directory
    for file in file_list:
        file_id = file['id']
        file_name = file['title']
        file_path = os.path.join(destination, file_name)
        print(f"Downloading {file_name}...")
        downloaded_file = drive.CreateFile({'id': file_id})
        downloaded_file.GetContentFile(file_path)
    print("All files downloaded successfully!")

def list_files_in_directory(directory):
    """
    List all files in a directory and print their names.

    :param directory: str, The directory to list files
    """
    file_names = os.listdir(directory)
    print(f"Files in {directory}:")
    for file_name in file_names:
        print(file_name)
    return file_names

In [ ]:
#@title **Step 5: Download Audio Files from Google Drive**
# This step will download all audio files from the shared Google Drive folder to the local directory.

# Replace the folder ID with your shared folder's ID
folder_id = '1PAkwwWpQY5Y8vgvkmVYm5Z_oKBVfr3e_'  # Update this with your folder ID
dataDir = '/content/audio_files'

# Download files from Google Drive
download_files_from_drive(folder_id, dataDir)

# List all files in the directory to verify download
file_names = list_files_in_directory(dataDir)

All files downloaded successfully!
Files in /content/audio_files:
XC3_20210404_054400.mp3


In [ ]:
#@title **Step 6: Initialise BirdNET Analyzer and Process Audio Files**
# This step initialises the BirdNET-Analyzer model and processes the downloaded audio files.
# Initialise BirdNET-Analyzer model
analyzer = Analyzer()

# Define the coordinates and date for the recordings
coordinates = [51.7813, -2.5745]  # Replace with your coordinates (Latitude, Longitude)
recording_date = datetime(year=2021, month=4, day=4)

# Analyse Audio Files and Store Results
results = {}

for file_name in file_names:
    try:
        file_path = os.path.join(dataDir, file_name)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Skip non-audio files
        if not file_name.lower().endswith(('.wav', '.mp3', '.flac')):
            print(f"Skipping non-audio file: {file_name}")
            continue

        # Initialise a Recording object
        recording = Recording(
            analyzer,
            file_path,  # path to the recording file
            lat=coordinates[0],  # latitude of the recording location
            lon=coordinates[1],  # longitude of the recording location
            date=recording_date,  # the date of the recording
            min_conf=0.1,  # Set Minimum Confidence
        )

        # Analyze the recording
        recording.analyze()

        # Store the results
        results[file_name] = recording.detections

    except Exception as e:
        print(f"Error processing {file_name}: {e}")

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data
read_audio_data: complete, read  1464 chunks.
analyze_recording XC3_20210404_054400.mp3
recording has lon/lat
set_predicted_species_list_from_position
return_predicted_species_list
13
141 species loaded.


In [ ]:
#@title **Step 7: Convert Results to DataFrame and Save as CSV**
# Flatten the results dictionary and convert it to a DataFrame. Save the DataFrame as a CSV file.

flattened_data = []
for filename, records in results.items():
    for record in records:
        record['filename'] = filename
        flattened_data.append(record)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

# Define the path to save the CSV file
csv_file_path = '/content/BirdnetProject/Files/data/CSV_data/XC/XC3_1_20210404_054400.csv'
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"Results have been successfully converted to CSV and saved to {csv_file_path}")


Results have been successfully converted to CSV and saved to /content/BirdnetProject/Files/data/CSV_data/XC/XC3_1_20210404_054400.csv


In [ ]:
#@title **Step 8: Load and Verify CSV Data**
# Load the saved CSV file into a DataFrame to verify the contents

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Print the first few rows of the DataFrame to verify the data
print(df.head())

            common_name      scientific_name  start_time  end_time  \
0        European Robin   Erithacus rubecula       195.0     198.0   
1             Goldcrest      Regulus regulus       273.0     276.0   
2      Common Firecrest  Regulus ignicapilla       423.0     426.0   
3  Eurasian Treecreeper   Certhia familiaris       447.0     450.0   
4    Common Wood-Pigeon     Columba palumbus       555.0     558.0   

   confidence                                    label  \
0    0.103502        Erithacus rubecula_European Robin   
1    0.175837                Regulus regulus_Goldcrest   
2    0.103631     Regulus ignicapilla_Common Firecrest   
3    0.271523  Certhia familiaris_Eurasian Treecreeper   
4    0.110816      Columba palumbus_Common Wood-Pigeon   

                  filename  
0  XC3_20210404_054400.mp3  
1  XC3_20210404_054400.mp3  
2  XC3_20210404_054400.mp3  
3  XC3_20210404_054400.mp3  
4  XC3_20210404_054400.mp3  


In [ ]:
#@title **Step 9: Save CSV File to Desired Location**
# This step allows users to save the CSV file to their desired location.

# Ask the user where they want to save the CSV file
save_option = input("Where would you like to save the CSV file? Enter 'local' to download to your device or 'drive' to save to Google Drive: ")

if save_option.lower() == 'drive':
    # Mount Google Drive
    drive.mount('/content/drive')
    drive_csv_path = input("Enter the path in your Google Drive where you want to save the CSV file (e.g., /content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/): ")
    drive_csv_path = os.path.join(drive_csv_path, os.path.basename(csv_file_path))
    # Copy the file to Google Drive
    !cp {csv_file_path} {drive_csv_path}
    print(f"CSV file saved to your Google Drive at {drive_csv_path}")
elif save_option.lower() == 'local':
    # Download the file to the user's local device
    files.download(csv_file_path)
    print("CSV file downloaded to your local device.")
else:
    print("Invalid option. Please enter 'local' or 'drive'.")

Where would you like to save the CSV file? Enter 'local' to download to your device or 'drive' to save to Google Drive: local


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV file downloaded to your local device.


In [ ]:
#@title **<------------ Part 2 ------------>**

In [ ]:
#@title **Creating Merged Dataframe Files**

In [ ]:
#@title **Step 1: Change Working Directory**
# Change working directory to defined path
os.chdir('/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/XC')

# Verify working directory
print(os.getcwd())

/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/XC


In [ ]:
#@title **Step 2: Create Merged Dataframe File For All Transects**

# Read in the CSV files
df_XC1 = pd.read_csv('XC1_25_20230615_180000.csv')
df_XC2 = pd.read_csv('XC2_25_20200530_171400.csv')
df_XC3 = pd.read_csv('XC3_25_20210404_054400.csv')

# Add a column to each dataframe to indicate the transect source
df_XC1['transect'] = 1
df_XC2['transect'] = 2
df_XC3['transect'] = 3

# Function to extract date and hour from the filename
def extract_datetime(filename):
    date_str = filename.split('_')[1]
    time_str = filename.split('_')[2].split('.')[0]
    datetime_str = date_str + ' ' + time_str
    datetime = pd.to_datetime(datetime_str, format='%Y%m%d %H%M%S')
    return datetime

# Apply the function to create a new datetime column
df_XC1['datetime'] = df_XC1['filename'].apply(extract_datetime)
df_XC2['datetime'] = df_XC2['filename'].apply(extract_datetime)
df_XC3['datetime'] = df_XC3['filename'].apply(extract_datetime)

# Function to convert seconds to hour:minutes:seconds format based on the datetime
def seconds_to_hms(datetime, seconds):
    new_time = datetime + pd.to_timedelta(seconds, unit='s')
    return new_time.strftime('%-H:%M:%S')

# Apply the function to convert start_time and end_time
df_XC1['start_time'] = df_XC1.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df_XC1['end_time'] = df_XC1.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df_XC2['start_time'] = df_XC2.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df_XC2['end_time'] = df_XC2.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)
df_XC3['start_time'] = df_XC3.apply(lambda row: seconds_to_hms(row['datetime'], row['start_time']), axis=1)
df_XC3['end_time'] = df_XC3.apply(lambda row: seconds_to_hms(row['datetime'], row['end_time']), axis=1)

# Concatenate the dataframes
concatenated_df = pd.concat([df_XC1, df_XC2], ignore_index=True)

# Extract date and start hour
concatenated_df['date'] = concatenated_df['datetime'].dt.date
concatenated_df['start_hour'] = concatenated_df['datetime'].dt.hour

# Sort the dataframe by transect, date, start_time, common_name, and confidence
concatenated_df = concatenated_df.sort_values(by=['transect', 'date', 'start_time', 'common_name', 'confidence'])

# Reorder the columns to have transect, date, start_hour, time, and readable start/end times
reordered_columns = ['transect', 'date', 'start_hour', 'start_time', 'end_time', 'common_name', 'scientific_name', 'confidence', 'label', 'filename']
concatenated_df = concatenated_df[reordered_columns]

# Display the first few rows of the concatenated dataframe
print("\nNew Dataframe:")
print(concatenated_df.head())
print(concatenated_df.tail())



New Dataframe:
   transect        date  start_hour start_time  end_time         common_name  \
0         1  2023-06-15          18   18:00:03  18:00:06   Eurasian Blackcap   
1         1  2023-06-15          18   18:00:06  18:00:09   Eurasian Blackcap   
2         1  2023-06-15          18   18:00:12  18:00:15   Eurasian Blackcap   
3         1  2023-06-15          18   18:00:18  18:00:21  Eurasian Blackbird   
4         1  2023-06-15          18   18:00:21  18:00:24   Eurasian Blackcap   

      scientific_name  confidence                                 label  \
0  Sylvia atricapilla    0.490236  Sylvia atricapilla_Eurasian Blackcap   
1  Sylvia atricapilla    0.586808  Sylvia atricapilla_Eurasian Blackcap   
2  Sylvia atricapilla    0.568825  Sylvia atricapilla_Eurasian Blackcap   
3       Turdus merula    0.417446      Turdus merula_Eurasian Blackbird   
4  Sylvia atricapilla    0.774901  Sylvia atricapilla_Eurasian Blackcap   

                  filename  
0  XC1_20230615_180000.

In [ ]:
#@title **Step 3: Decide Where to Save Files**
# Ask the user where they want to save the CSV and Excel files
save_option = input("Where would you like to save the files? Enter 'local' to download to your device or 'drive' to save to Google Drive: ")

if save_option.lower() == 'drive':
    # Mount Google Drive
    drive.mount('/content/drive')
    drive_path = input("Enter the path in your Google Drive where you want to save the files (e.g., /content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/): ")
    csv_output_path = os.path.join(drive_path, 'XC_25_Merged.csv')
    excel_output_path = os.path.join(drive_path, 'XC_25_Merged.xlsx')
    concatenated_df.to_csv(csv_output_path, index=False)
    concatenated_df.to_excel(excel_output_path, index=False)
    print(f"CSV file saved to your Google Drive at {csv_output_path}")
    print(f"Excel file saved to your Google Drive at {excel_output_path}")
elif save_option.lower() == 'local':
    # Download the files to the user's local device
    csv_output_path = 'XC_25_Merged.csv'
    excel_output_path = 'XC_25_Merged.xlsx'
    concatenated_df.to_csv(csv_output_path, index=False)
    concatenated_df.to_excel(excel_output_path, index=False)
    files.download(csv_output_path)
    files.download(excel_output_path)
    print("CSV and Excel files downloaded to your local device.")
else:
    print("Invalid option. Please enter 'local' or 'drive'.")

Where would you like to save the files? Enter 'local' to download to your device or 'drive' to save to Google Drive: local


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV and Excel files downloaded to your local device.
